In [1]:
import pandas as pd

In [9]:
jd_raw = pd.read_excel("data.xlsx",sheet_name="京东",header=1)

In [13]:
jd_raw.head(3)

,ID,name,comment_num,shop_name,link,commentVersion,score1count,score2count,score3count,score4count,score5count,price
0,1127466,↵ 澳大利亚 进口牛奶 德运（Devondale） 全脂牛奶 ...,185238,德运（Devondale）,//item.jd.com/1127466.html,109576.0,1335,478,1646,6451,175328,99.0
1,2118943,↵ 澳大利亚 进口牛奶 哈威鲜（(Harvery fresh)...,58513,哈威鲜,//item.jd.com/2118943.html,46780.0,323,130,369,1568,56123,99.0
2,1805138,↵ 澳大利亚 进口奶粉 德运 （Devondale）调制成人奶...,16181,德运（Devondale）,//item.jd.com/1805138.html,18314.0,169,65,144,452,15351,59.9


In [17]:
dd_raw = pd.read_excel("data.xlsx", sheet_name = "多点", header = 0)

In [18]:
dd_raw.head()

,类别一,类别二,品牌名字,Unnamed: 3,名称,价格,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,奶饮熟食,进口奶品,丸岛,1,[次日达] 麦可超高温全脂灭菌乳 1L 波兰,7.9,https://img.dmallcdn.com/20170921/b19c4fc2-2e2...,全球精选,牛奶乳制品,精选进口牛奶乳饮
1,奶饮熟食,进口奶品,侬家,2,[次日达] 法国侬家牌超高温灭菌低脂牛奶 1L,7.9,https://img.dmallcdn.com/20170515/6c7c3f87-39f...,全球精选,牛奶乳制品,精选进口牛奶乳饮
2,奶饮熟食,进口奶品,丸岛,3,[次日达] 纽仕兰全脂牛奶 250ml*10 新西兰,99.0,https://img.dmallcdn.com/20170817/e3220715-125...,全球精选,牛奶乳制品,精选进口牛奶乳饮
3,奶饮熟食,进口奶品,丸岛,4,[次日达] 侬家全脂牛奶 超高温灭菌 1L 法国,9.6,https://img.dmallcdn.com/20161215/8dffc586-554...,全球精选,牛奶乳制品,精选进口牛奶乳饮
4,奶饮熟食,进口奶品,萨克森,5,[次日达] 萨克森全脂高温杀菌奶 3.5%脂肪含量 1L*12 德国,118.8,https://img.dmallcdn.com/20170921/9ef80d90-cb2...,全球精选,牛奶乳制品,精选进口牛奶乳饮


In [19]:
import jieba

In [65]:
import re

In [30]:
dd_name = dd_raw.loc[:,"名称"]

In [54]:
jieba.add_word("麦可")
jieba.add_word("侬家")
jieba.add_word("次日达")
jieba.add_word("发酵乳")
jieba.add_word("灭菌乳")

In [63]:
project = {"纯牛奶":"牛奶","奶":"牛奶"}

In [316]:
p = []
Count = []
Net = []
for line in dd_name:
    seg = jieba.cut(line, cut_all = False)
    net = 0
    count = 1
    attr = re.findall(r"[0-9]+[a-zA-Z]+\*[0-9]+",line)
    if(attr):
        n,c = attr[0].split("*")
        net = int(re.match(r"[0-9]*",n).group())
        count = int(c)
#         if(re.search)
    else:
        attr = re.findall(r"\d+[mlLg]+",line)
        net = int(re.match(r"[0-9]*",attr[0]).group())
    unit = re.search(r"[mlLg]+",attr[0]).group()

    if(unit == 'L'):
        net = net*1000
    if(unit == 'g'):
        net = 0-net
    Count.append(count)
    Net.append(net)
    for item in seg:     
        if(not re.match(r"^\D+$",item) or item == " " or item == "*"):
            continue
        if(item not in ["[","]","次日达","含量",'灭菌','上质','杀菌']):  
            try:
                p.append(project[item])
            except KeyError:
                p.append(item)
feature = list(set(p))

In [317]:
feature

['酸牛奶',
 '高温',
 '侬家',
 '瑞士',
 '法国',
 '麦可',
 '灭菌乳',
 '艾美',
 '芒果',
 '礼盒装',
 '全脂',
 '波兰',
 '牛奶',
 '德亚',
 '发酵乳',
 '脂肪',
 '脱脂',
 '超高温',
 '低脂',
 '安佳',
 '新西兰',
 '草莓',
 '纽仕兰',
 '澳大利亚',
 '侬家牌',
 '萨克森',
 '德国']

In [318]:
txtFeature = []
for line in dd_name:
    seg = jieba.cut(line, cut_all = False)
    f = [0 for i in feature]
    for i in seg:
        try:
            f[feature.index(i)] = 1
        except ValueError:
            continue
    txtFeature.append(f)

In [319]:
dd_hash = pd.DataFrame(txtFeature)

In [320]:
dd_hash["count"] = Count
dd_hash["net"] = Net

In [321]:
dd_hash.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,count,net
0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1000
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1000
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,10,250
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1000
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,12,1000


In [322]:
jd_name = jd_raw["name"]

In [323]:
Count = []
Net = []
txtFeature = []
for line in jd_name:
    net = 0
    count = 1
    attr = re.search(r"[0-9]+[a-zA-Z]+\*[0-9]+",line)
    if(attr):
        n,c = attr.group().split("*")
        net = int(re.match(r"[0-9]*",n).group())
        count = int(c)
    else:
        attr = re.search(r"\d+[mlLg]+",line)
        if(attr):
            net = int(re.match(r"[0-9]*",attr.group()).group())
    if(attr):
        unit = re.search(r"[mlLgKGk]+",attr.group()).group()
        unit = unit.lower()
        if(unit == 'l'):
            net = net * 1000
        if(unit == 'g'):
            net = 0 - net
        if(unit == 'kg'):
            net = 0 - net * 1000
    Net.append(net)
    Count.append(count)
    seg = jieba.cut(line, cut_all=False)
    f = [0 for i in feature]
    for i in seg:
        try:
            f[feature.index(i)] = 1
        except ValueError:
            continue
    txtFeature.append(f)

In [324]:
jd_hash = pd.DataFrame(txtFeature)

In [325]:
jd_hash["count"] = Count
jd_hash["net"] = Net

In [326]:
jd_hash.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,count,net
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,10,1000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,12,1000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,12,1000
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,24,250


In [327]:
import numpy as np

In [328]:
max_count = max(max(jd_hash["count"]),max(dd_hash["count"]))
max_net = max(max(jd_hash["net"]),max(dd_hash["net"]))
min_net = min(min(jd_hash["net"]),min(dd_hash["net"]))

In [329]:
max_net,min_net

(250000, -1600)

In [330]:
jd_hash["count"] = (jd_hash["count"]-1)/(max_count - 1)
jd_hash.loc[jd_hash["net"] > 0,"net"] = np.abs(jd_hash.loc[jd_hash["net"] > 0,"net"])/max_net
jd_hash.loc[jd_hash["net"] < 0,"net"] = jd_hash.loc[jd_hash["net"] < 0,"net"]/np.abs(min_net)

In [332]:
jd_hash.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,count,net
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.191489,0.004
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.234043,0.004
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.000000,0.000
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0.234043,0.004
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0.489362,0.001


In [333]:
dd_hash["count"] = (dd_hash["count"]-1)/(max_count - 1)
dd_hash.loc[dd_hash["net"] > 0,"net"] = np.abs(dd_hash.loc[dd_hash["net"] > 0,"net"])/max_net
dd_hash.loc[dd_hash["net"] < 0,"net"] = dd_hash.loc[dd_hash["net"] < 0,"net"]/np.abs(min_net)

In [335]:
dd_hash.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,count,net
0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0.004
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0.000000,0.004
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0.191489,0.001
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0.004
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0.234043,0.004


In [340]:
distance = []
for i in range(jd_hash.shape[0]):
    centroid = -1
    dist = 10000
    for j in range(dd_hash.shape[0]):
        d = ((jd_hash.iloc[i,:] - dd_hash.iloc[j,:])**2).sum()
        if(d<dist):
            dist = d
            centroid = j
    distance.append((i,dist,centroid))

In [355]:
for i in distance:
    if(i[1]<0.01):
        print(f"京东编号：{jd_raw.iloc[i[0],0]}")
        print(f"京东名称：{jd_raw.iloc[i[0],1]}")
        print(f"多点编号：{dd_raw.iloc[i[2],3]}  多点名称：{dd_raw.iloc[i[2],4]}  匹配度：{(1-i[1])*100}%\n")


京东编号：896023
京东名称：↵                德国进口牛奶 德亚（Weidendorf）脱脂 1L*12      
多点编号：12  多点名称：[次日达] 德亚脱脂牛奶 200ml*12 德国  匹配度：99.998976%

京东编号：2149294
京东名称：↵                德国进口牛奶 德亚（Weidendorf）脱脂 200ml*30      
多点编号：9  多点名称：[次日达] 德亚脱脂牛奶 200ml*30 德国  匹配度：100.0%

京东编号：896024
京东名称：↵                德国进口牛奶 德亚（Weidendorf）全脂 200ml*30      
多点编号：7  多点名称：[次日达] 德亚全脂牛奶 200ml*30 德国  匹配度：100.0%

京东编号：2130957
京东名称：↵                德国进口牛奶 德亚（Weidendorf）低脂 200ml*30      
多点编号：16  多点名称：[次日达] 德亚低脂牛奶 200ml*30 德国  匹配度：100.0%

京东编号：3746501
京东名称：↵                新西兰进口牛奶 安佳Anchor卓冉有机纯牛奶250ml*10礼盒装      
多点编号：11  多点名称：[次日达] 安佳牛奶礼盒装 250ml*10 新西兰  匹配度：100.0%

京东编号：1546818
京东名称：↵                新西兰进口儿童牛奶 安佳Anchor金装儿童牛奶190ml*12 礼盒装      
多点编号：11  多点名称：[次日达] 安佳牛奶礼盒装 250ml*10 新西兰  匹配度：99.81891682940697%

京东编号：10500420833
京东名称：↵                中粮我买网 上质 脱脂纯牛奶 1L*12（法国进口）      
多点编号：8  多点名称：[次日达] 上质脱脂纯牛奶 1L*12 法国  匹配度：100.0%

京东编号：1570432685
京东名称：↵                【中粮我买网】德国进口牛奶 德亚（Weidendorf）全脂牛奶 200ml*30      
多点编号：7  多点名称：[次日达]

## If want higher accuracy, increase the weight of `count` and `net`, 
## and decrease the threshold

In [359]:
dd_hash["count"] = dd_hash["count"]*2
dd_hash["net"] = dd_hash["net"]*5
jd_hash["count"] = jd_hash["count"]*2
jd_hash["net"] = jd_hash["net"]*5

In [360]:
distance = []
for i in range(jd_hash.shape[0]):
    centroid = -1
    dist = 10000
    for j in range(dd_hash.shape[0]):
        d = ((jd_hash.iloc[i,:] - dd_hash.iloc[j,:])**2).sum()
        if(d<dist):
            dist = d
            centroid = j
    distance.append((i,dist,centroid))

In [363]:
for i in distance:
    if(i[1]<0.0005):
        print(f"京东编号：{jd_raw.iloc[i[0],0]}")
        print(f"京东名称：{jd_raw.iloc[i[0],1]}")
        print(f"多点编号：{dd_raw.iloc[i[2],3]}  多点名称：{dd_raw.iloc[i[2],4]}  匹配度：{(1-i[1])*100}%\n")

京东编号：2149294
京东名称：↵                德国进口牛奶 德亚（Weidendorf）脱脂 200ml*30      
多点编号：9  多点名称：[次日达] 德亚脱脂牛奶 200ml*30 德国  匹配度：100.0%

京东编号：896024
京东名称：↵                德国进口牛奶 德亚（Weidendorf）全脂 200ml*30      
多点编号：7  多点名称：[次日达] 德亚全脂牛奶 200ml*30 德国  匹配度：100.0%

京东编号：2130957
京东名称：↵                德国进口牛奶 德亚（Weidendorf）低脂 200ml*30      
多点编号：16  多点名称：[次日达] 德亚低脂牛奶 200ml*30 德国  匹配度：100.0%

京东编号：3746501
京东名称：↵                新西兰进口牛奶 安佳Anchor卓冉有机纯牛奶250ml*10礼盒装      
多点编号：11  多点名称：[次日达] 安佳牛奶礼盒装 250ml*10 新西兰  匹配度：100.0%

京东编号：10500420833
京东名称：↵                中粮我买网 上质 脱脂纯牛奶 1L*12（法国进口）      
多点编号：8  多点名称：[次日达] 上质脱脂纯牛奶 1L*12 法国  匹配度：100.0%

京东编号：1570432685
京东名称：↵                【中粮我买网】德国进口牛奶 德亚（Weidendorf）全脂牛奶 200ml*30      
多点编号：7  多点名称：[次日达] 德亚全脂牛奶 200ml*30 德国  匹配度：100.0%

京东编号：1576588328
京东名称：↵                【中粮我买网】 德亚 全脂牛奶 200ml*30（德国进口）      
多点编号：7  多点名称：[次日达] 德亚全脂牛奶 200ml*30 德国  匹配度：100.0%

京东编号：16892104985
京东名称：↵                德国进口牛奶德亚全脂牛奶常温保存营养饮料饮品中秋节送礼休闲零食组合大礼包 德亚全脂纯牛奶200ml*30盒      
多点编号：7  多点名称